In [ ]:
import pandas as pd

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
df = pd.read_csv('caminho/base.csv')

In [ ]:
df

In [ ]:
# Verificar o tipo da coluna 'date'
print(df['date'].dtype)

In [ ]:
# Se não estiver no formato datetime, converta-a
df['date'] = pd.to_datetime(df['date'])

In [ ]:
# Substituir os valores na coluna gptpt
df['gptpt'] = df['gptpt'].replace({-1: 'Negativo', 0: 'Neutro', 1: 'Positivo'})

In [ ]:
df['Marca'] = df['Marca'].str.upper()

In [ ]:
df['Produto'] = df['Produto'].str.upper()

In [ ]:
print(df.Marca.value_counts())

In [ ]:
df.rename(columns={'gptpt': 'Sentimento'}, inplace=True)

Gráficos de Polaridade: Crie gráficos de barras ou gráficos de pizza para mostrar a distribuição de sentimentos (positivo, neutro, negativo) por marca e por produto.

In [ ]:
# Agrupar por Marca e gptpt (polaridade)
sentimento_marca = df.groupby(['Marca', 'Sentimento']).size().unstack().fillna(0)

# Ajustar o tamanho do gráfico
plt.figure(figsize=(12, 8))  # Aumentar o tamanho do gráfico
sentimento_marca.plot(kind='bar', stacked=True, figsize=(12, 8))
plt.title('Distribuição de Sentimentos por Marca')
plt.xlabel('Marca')
plt.ylabel('Quantidade')
plt.xticks(rotation=45, ha='right')  # Rotacionar os nomes para facilitar a leitura
plt.show()

# Criar gráfico de pizza para cada Marca
for marca in sentimento_marca.index:
    sentimento_marca.loc[marca].plot(kind='pie', autopct='%1.1f%%', startangle=90)
    plt.title(f'Distribuição de Sentimentos - {marca}')
    plt.ylabel('')  # Remover label desnecessária
    plt.show()


In [ ]:
# Agrupar por Produto e gptpt (polaridade)
sentimento_produto = df.groupby(['Produto', 'Sentimento']).size().unstack().fillna(0)

# Criar gráfico de barras
plt.figure(figsize=(12, 8))  # Aumentar o tamanho do gráfico
sentimento_produto.plot(kind='bar', stacked=True, figsize=(12, 8))
plt.title('Distribuição de Sentimentos por Produto')
plt.xlabel('Produto')
plt.ylabel('Quantidade')
plt.xticks(rotation=45, ha='right')  # Rotacionar os nomes para facilitar a leitura
plt.show()

# Criar gráfico de pizza para cada Produto
for produto in sentimento_produto.index:
    sentimento_produto.loc[produto].plot(kind='pie', autopct='%1.1f%%', startangle=90)
    plt.title(f'Distribuição de Sentimentos - {produto}')
    plt.ylabel('')  # Remover label desnecessária
    plt.show()


Tendências de Sentimentos: Mostre a evolução dos sentimentos ao longo do tempo para cada marca. Use gráficos de linha para destacar períodos com aumento de sentimentos negativos ou positivos.

In [ ]:
# Agrupar por Marca, data e sentimento
sentimento_tempo = df.groupby([df['date'].dt.to_period('M'), 'Marca', 'Sentimento']).size().unstack().fillna(0)

# Criar gráfico de linha para cada Marca
for marca in df['Marca'].unique():
    sentimento_tempo_marca = sentimento_tempo.xs(marca, level='Marca')

    # Plotar gráfico de linha
    ax = sentimento_tempo_marca.plot(kind='line', figsize=(12, 8), marker='o')
    plt.title(f'Evolução dos Sentimentos ao Longo do Tempo - {marca}')
    plt.xlabel('Tempo')
    plt.ylabel('Quantidade')
    plt.xticks(rotation=45)
    plt.grid(True)

    # Adicionar rótulos nos pontos do gráfico
    for i in range(len(sentimento_tempo_marca)):
        for sentimento in sentimento_tempo_marca.columns:
            y = sentimento_tempo_marca.iloc[i][sentimento]
            if y > 0:  # Exibir apenas para pontos com valores positivos
                ax.text(sentimento_tempo_marca.index[i], y, f'{int(y)}', ha='center', va='bottom')

    # Garantir que os valores do eixo Y sejam inteiros
    ax.yaxis.get_major_locator().set_params(integer=True)

    plt.show()

Gráficos Temporais: Crie gráficos que mostrem o número de avaliações por mês ou trimestre. Isso pode ajudar a identificar picos de avaliações, que podem estar associados a campanhas de marketing ou lançamentos de produtos.

Correlacionar com Campanhas: Se possível, correlacione esses picos com campanhas de marketing ou eventos sazonais.

In [ ]:
# Agrupar por marca e trimestre
avaliacoes_por_trimestre_marca = df.groupby([df['date'].dt.to_period('Q'), 'Marca']).size().unstack().fillna(0)

# Criar gráfico de linha para cada marca mostrando avaliações por trimestre
for marca in avaliacoes_por_trimestre_marca.columns:
    plt.figure(figsize=(12, 8))
    ax = avaliacoes_por_trimestre_marca[marca].plot(kind='line', marker='o')
    plt.title(f'Número de Avaliações por Trimestre - {marca}')
    plt.xlabel('Trimestre')
    plt.ylabel('Número de Avaliações')
    plt.xticks(rotation=45)
    plt.grid(True)

    # Adicionar rótulos nos pontos do gráfico
    for i in range(len(avaliacoes_por_trimestre_marca)):
        y = avaliacoes_por_trimestre_marca[marca].iloc[i]
        if y > 0:  # Exibir apenas para pontos com valores positivos
            ax.text(avaliacoes_por_trimestre_marca.index[i], y, f'{int(y)}', ha='center', va='bottom')

    # Garantir que os valores do eixo Y sejam inteiros
    ax.yaxis.get_major_locator().set_params(integer=True)

    plt.show()

Análise de Sentimentos Negativos: Faça uma análise mais profunda das avaliações negativas para identificar problemas comuns mencionados pelos clientes. Use nuvens de palavras para visualizar as principais queixas.

Oportunidades de Melhoria: Baseado nas avaliações negativas e neutras, sugira melhorias para produtos específicos.

In [ ]:
# Filtrar as avaliações negativas
df_negativas = df[df['Sentimento'] == 'Negativo']


In [ ]:
# Remover stopwords do texto
import nltk
nltk.download('stopwords')
stopwords_portugues = set(nltk.corpus.stopwords.words('portuguese'))
stopwords_portugues.update(["produto", "marca", "comprei", "cerveja", "comprar", "Amazon"])


In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

# Loop através de cada marca única
for marca in df_negativas['Marca'].unique():
    # Filtrar as avaliações negativas para a marca atual
    df_negativa_marca = df_negativas[df_negativas['Marca'] == marca]

    # Unir todos os textos das avaliações negativas dessa marca em uma única string
    texto_negativo_marca = " ".join(df_negativa_marca['review'].astype(str))

    # Limpar o texto removendo stopwords
    texto_limpo_marca = " ".join([word for word in texto_negativo_marca.split() if word.lower() not in stopwords_portugues])

    # Gerar a nuvem de palavras para a marca
    wordcloud = WordCloud(width=800, height=400, background_color='white', stopwords=stopwords_portugues).generate(texto_limpo_marca)

    # Mostrar a nuvem de palavras
    plt.figure(figsize=(10, 5))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis('off')
    plt.title(f'Principais Queixas em Avaliações Negativas - {marca}')
    plt.show()


In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from collections import Counter
import string
import nltk
from nltk.corpus import stopwords

# Se ainda não tiver baixado as stopwords
nltk.download('stopwords')

# Filtrar avaliações negativas
df_negativas = df[df['Sentimento'] == 'Negativo']

# Preprocessamento do texto
stop_words = set(stopwords.words('portuguese'))
stop_words.update(["produto", "marca", "comprei", "cerveja", "comprar", "Amazon"])
def preprocess_text(text):
    text = text.lower()  # Converte para minúsculas
    text = text.translate(str.maketrans('', '', string.punctuation))  # Remove pontuação
    words = text.split()  # Divide em palavras
    words = [word for word in words if word not in stop_words]  # Remove stopwords
    return ' '.join(words)

df_negativas['review_clean'] = df_negativas['review'].apply(preprocess_text)

# Gerar nuvens de palavras para cada loja e marca
lojas = df_negativas['loja'].unique()

for loja in lojas:
    df_loja = df_negativas[df_negativas['loja'] == loja]
    marcas = df_loja['Marca'].unique()

    for marca in marcas:
        df_marca = df_loja[df_loja['Marca'] == marca]
        texto = ' '.join(df_marca['review_clean'])

        # Gera a nuvem de palavras
        wordcloud = WordCloud(width=800, height=400, background_color='white').generate(texto)

        # Mostrar a nuvem de palavras
        plt.figure(figsize=(10, 5))
        plt.imshow(wordcloud, interpolation='bilinear')
        plt.title(f'Nuvem de Palavras - Loja: {loja} - Marca: {marca}')
        plt.axis('off')
        plt.show()


Avaliações Positivas: Destaque produtos e marcas que têm uma alta proporção de avaliações positivas. Isso pode ser usado para reforçar a comunicação de marketing e campanhas publicitárias.

In [ ]:
# Filtrar as avaliações positivas
df_positivas = df[df['Sentimento'] == 'Positivo']

In [ ]:
# Remover stopwords do texto
import nltk
nltk.download('stopwords')
stopwords_portugues = set(nltk.corpus.stopwords.words('portuguese'))
stopwords_portugues.update(["produto", "marca", "comprei", "cerveja", "comprar", "Amazon"])


In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

# Loop através de cada marca única
for marca in df_positivas['Marca'].unique():
    # Filtrar as avaliações positivas para a marca atual
    df_positivas_marca = df_positivas[df_positivas['Marca'] == marca]

    # Unir todos os textos das avaliações positivas dessa marca em uma única string
    texto_positivo_marca = " ".join(df_positivas_marca['review'].astype(str))

    # Limpar o texto removendo stopwords
    texto_limpo_marca = " ".join([word for word in texto_positivo_marca.split() if word.lower() not in stopwords_portugues])

    # Gerar a nuvem de palavras para a marca
    wordcloud = WordCloud(width=800, height=400, background_color='white', stopwords=stopwords_portugues).generate(texto_limpo_marca)

    # Mostrar a nuvem de palavras
    plt.figure(figsize=(10, 5))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis('off')
    plt.title(f'Principais Queixas em Avaliações Positivas - {marca}')
    plt.show()

In [ ]:
# Contar o número total de avaliações por marca
marcas_com_mais_avaliacoes = df['Marca'].value_counts().head(10)


In [ ]:
# Filtrar o DataFrame para incluir apenas as 15 marcas com mais avaliações
df_top_marcas = df[df['Marca'].isin(marcas_com_mais_avaliacoes.index)]

In [ ]:
# Filtrar as avaliações positivas entre as 15 marcas
df_positivas_top_marcas = df_top_marcas[df_top_marcas['Sentimento'] == 'Positivo']

# Calcular a proporção de avaliações positivas para cada uma das 15 marcas
proporcao_positivas_top_marcas = df_positivas_top_marcas['Marca'].value_counts() / df_top_marcas['Marca'].value_counts()

# Ordenar pela proporção de avaliações positivas (da maior para a menor)
proporcao_positivas_top_marcas = proporcao_positivas_top_marcas.sort_values(ascending=False)

In [ ]:
# Gráfico de barras para as 15 marcas com mais avaliações
plt.figure(figsize=(12, 7))
proporcao_positivas_top_marcas.plot(kind='bar', color='green')
plt.title('Proporção de Avaliações Positivas nas 10 Marcas com Mais Avaliações')
plt.xlabel('Marca')
plt.ylabel('Proporção de Avaliações Positivas')
plt.xticks(rotation=45)
plt.show()
